### Q7b)

In [1]:
# Imports
import pandas as pd

In [2]:
flow_data = pd.read_csv('../../datasets/flows.csv')
flow_data

,day,month,year,flow_m3_s
0,1,1,1970,29.360
1,2,1,1970,28.880
2,3,1,1970,27.990
3,4,1,1970,23.610
4,5,1,1970,21.950
...,...,...,...,...
11319,28,12,2000,33.600
11320,29,12,2000,31.420
11321,30,12,2000,28.970
11322,31,12,2000,31.300


### Check for missing:
* any missing data
* any erroneous data (e.g. negative, or large values which are clearly incorrect)
* any samples which are too small or not matching in size if they are to be compared against other samples


In [3]:
# using isnull() function
flow_data.isnull()

,day,month,year,flow_m3_s
0,False,False,False,False
1,False,False,False,False
2,False,False,False,False
3,False,False,False,False
4,False,False,False,False
...,...,...,...,...
11319,False,False,False,False
11320,False,False,False,False
11321,False,False,False,False
11322,False,False,False,False


In [4]:
# Any missing values?
flow_data.isnull().values.any()

True

In [5]:
print(flow_data.isnull().values.sum())

6


### Remove missing values

In [6]:
flow_data = flow_data.dropna()
print(flow_data.isnull().values.sum())

0


### Copy data

In [7]:
flow_data_copy = flow_data.copy()

### Transform the data to time series format

In [8]:
flow_data_copy['dateInt'] = flow_data_copy['year'].astype(str) + flow_data_copy['month'].astype(str).str.zfill(2)+ flow_data_copy['day'].astype(str).str.zfill(2)
flow_data_copy['Date'] = pd.to_datetime(flow_data_copy['dateInt'], format='%Y%m%d')
flow_data_copy

,day,month,year,flow_m3_s,dateInt,Date
0,1,1,1970,29.360,19700101,1970-01-01
1,2,1,1970,28.880,19700102,1970-01-02
2,3,1,1970,27.990,19700103,1970-01-03
3,4,1,1970,23.610,19700104,1970-01-04
4,5,1,1970,21.950,19700105,1970-01-05
...,...,...,...,...,...,...
11319,28,12,2000,33.600,20001228,2000-12-28
11320,29,12,2000,31.420,20001229,2000-12-29
11321,30,12,2000,28.970,20001230,2000-12-30
11322,31,12,2000,31.300,20001231,2000-12-31


In [9]:
monthly_flow_data_series =  flow_data_copy.filter(['Date', 'flow_m3_s'], axis=1)
monthly_flow_data_series = monthly_flow_data_series.set_index('Date').resample('M').sum()
monthly_flow_data_series

,flow_m3_s
Date,
1970-01-31,2147.320
1970-02-28,2032.020
1970-03-31,1112.220
1970-04-30,1918.950
1970-05-31,649.940
...,...
2000-09-30,2051.640
2000-10-31,3389.370
2000-11-30,4448.620


### Calculate the observed data flow using formula (given in the question description)

In [27]:
monthly_flow_data_series.groupby([monthly_flow_data_series['Date'].dt.year, monthly_flow_data_series['Date'].dt.month, monthly_flow_data_series['Date'].dt.week]).agg(['sum', 'mean', 'std'])

/tmp/ipykernel_166518/1774716885.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  monthly_flow_data_series.groupby([monthly_flow_data_series['Date'].dt.year, monthly_flow_data_series['Date'].dt.month, monthly_flow_data_series['Date'].dt.week]).agg(['sum', 'mean', 'std'])


flow_m3_s              
                     sum      mean std
Date Date Date                        
1970 1    5     2147.320  2147.320 NaN
     2    9     2032.020  2032.020 NaN
     3    14    1112.220  1112.220 NaN
     4    18    1918.950  1918.950 NaN
     5    22     649.940   649.940 NaN
...                  ...       ...  ..
2000 9    39    2051.640  2051.640 NaN
     10   44    3389.370  3389.370 NaN
     11   48    4448.620  4448.620 NaN
     12   52    4292.590  4292.590 NaN
2001 1    5       72.836    72.836 NaN

[373 rows x 3 columns]

In [19]:
int(monthly_flow_data_series.Date.dt.strftime('%d')[0])

31

In [29]:
u_t = monthly_flow_data_series.flow_m3_s.mean()
u_t

1563.326249329759

In [30]:
a_t = monthly_flow_data_series.flow_m3_s.std()
a_t

1129.7573489097852

### Correlation for monthly series.

In [32]:
#monthly_rainfall_data_series = monthly_rainfall_data_series.squeeze(axis=0)
monthly_flow_data_series['flow_m3_s'].squeeze().autocorr(lag=2)

0.18050767385939198